In [22]:
import cropped_images

In [24]:
import numpy as np
from sklearn.manifold import TSNE

In [ ]:
X_train, y_train, X_test, y_test = cropped_images.load_data()

In [25]:
X_train_std = X_train / 255.0

In [28]:
import crop_encoder_data
#X_train are black and white images
#Y_train are the cropped images

In [29]:
#suffix of ed corresponds to encoder data
X_train_ed, y_train_ed, X_test_ed, y_test_ed = crop_encoder_data.load_data()

In [35]:
#suffix of edstd stands for ecoder data standardized
X_train_edstd = X_train_ed / 255.0

In [54]:
def flatten_dataset(X):
    num_img = X.shape[0]
    height = X.shape[1]
    width = X.shape[2]
    flat = []
    for i in range(num_img):
        img = X[i]
        sub_flat = []
        for j in range(len(img)):
            row = img[j]
            for k in range(len(row)):
                sub_flat.append(row[k])
        flat.append(sub_flat)
    return np.array(flat)

In [55]:
flat_X_train_edstd = flatten_dataset(X_train_edstd)

In [60]:
tsne = TSNE(n_components=2)
embed_X_train_edstd = tsne.fit_transform(flat_X_train_edstd)

In [61]:
import plotly as py 
py.tools.set_credentials_file(username='sunil.mall', api_key='NPtnMQ0tGFxgpsrkuccP') 
import plotly.plotly as py 
py.sign_in("sunil.mall", "NPtnMQ0tGFxgpsrkuccP") 
import plotly.graph_objs as go

In [62]:
red_x = [i[0] for i in embed_X_train_edstd]
red_y = [i[1] for i in embed_X_train_edstd]

In [77]:
np.array(red_x).shape

(2095,)

In [75]:
trace = go.Scatter(
    x = red_x,
    y = red_y,
    mode = 'markers',
    marker = dict(size=2),
    #text = flat_sub_y
)

data = [trace]

py.iplot(data, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunil.mall/0 or inside your plot.ly account where it is named 'basic-scatter'


In [66]:
tsne_2 = TSNE(n_)

(2095,)

In [67]:
y_train_edstd = y_train_ed / 255.0

In [68]:
y_train_edstd.shape

(2095, 64, 64)

In [69]:
flat_y_train_edstd = flatten_dataset(y_train_edstd)

In [70]:
flat_y_train_edstd.shape

(2095, 4096)

In [71]:
tsne_y = TSNE(n_components=2)
embed_y_train_edstd = tsne_y.fit_transform(flat_y_train_edstd)

In [72]:
crop_red_x = [i[0] for i in embed_y_train_edstd]
crop_red_y = [i[1] for i in embed_y_train_edstd]

In [76]:
np.array(crop_red_x).shape

(2095,)

In [74]:
trace_crop = go.Scatter(
    x = crop_red_x,
    y = crop_red_y,
    mode = 'markers',
    marker = dict(size=2),
    #text = flat_sub_y
)

data_crop = [trace_crop]

py.iplot(data_crop, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunil.mall/0 or inside your plot.ly account where it is named 'basic-scatter'


In [107]:
#crop_encoder_data.py
import numpy as np
def create_pickle():
    import os
    from PIL import Image
    import pickle
    np.random.seed(5)
    print "Creating Pickle"
    crop_folders = ['ALB', 'BET', 'DOL', 'LAG', 'SHARK', 'YFT']
    X = []
    Y = []
    labels = []
    img_data = {}
    for (label,folder) in enumerate(crop_folders):
        orig_file_names = os.listdir(os.getcwd()+"/train/cropped/"+folder+"/")
        X += map(lambda x: os.getcwd()+"/train/"+folder+"/"+x, orig_file_names)
        crop_file_names = map(lambda x: os.getcwd()+"/train/cropped/"+folder+"/"+x, orig_file_names)
        Y += crop_file_names
        labels += [label for x in xrange(len(crop_file_names))]


    #Creates Images from filepaths
    X = map(lambda f_name: np.asarray( Image.open(f_name).convert('L').resize((256,256)), dtype='uint8' ), X)
    Y = map(lambda f_name: np.asarray( Image.open(f_name).convert('L'), dtype='uint8' ), Y)
    X = np.asarray(X,dtype='uint8')
    Y = np.asarray(Y,dtype='uint8')
    labels = np.asarray(labels, dtype='uint8')
    labels.shape = (1,labels.shape[0])

    #Shuffles data
    X_train,y_train,X_test,y_test,labels_train,labels_test = test_train_split(X,Y,labels, 0.7)

    img_data['X_train'] = X_train
    img_data['y_train'] = y_train
    img_data['X_test'] = X_test
    img_data['y_test'] = y_test
    img_data['labels_train'] = labels_train
    img_data['labels_test'] = labels_test
    
    with open('encoder_data.pkl', 'wb') as my_pickle:
        pickle.dump(img_data,my_pickle)

def load_data(perc=0.7):
    import os
    if(os.path.isfile(os.getcwd()+"/encoder_data.pkl") == False):
        create_pickle()
    import pickle
    with open("encoder_data.pkl","rb") as img_f:
        data = pickle.load(img_f)
    return test_train_split(np.concatenate((data['X_train'],data['X_test']),axis=0),
                    np.concatenate((data['y_train'],data['y_train']),axis=0), np.concatenate((data['labels_train'],data['labels_train']), axis=1),perc)

def test_train_split(X_data,y_data,labels_data, percentage):
    perm = np.random.permutation(np.arange(X_data.shape[0]))
    X_data = X_data[perm,:]
    y_data = y_data[perm,:]
    labels_data = labels_data[:,perm]
    train_size = int(X_data.shape[0]*percentage)

    X_train = X_data[:train_size,:]
    y_train = y_data[:train_size,:]
    labels_train = labels_data[:,:train_size]
    
    X_test = X_data[train_size:,:]
    y_test = y_data[train_size:,:]
    labels_test = labels_data[:,train_size:]
    
    return (X_train,y_train,X_test,y_test, labels_train, labels_test)

In [109]:
#suffix of ed corresponds to encoder data
X_train_ed, y_train_ed, X_test_ed, y_test_ed, labels_train_ed, labels_test_ed = load_data()

(1, 4190)


In [110]:
#suffix of edstd stands for ecoder data standardized
X_train_edstd = X_train_ed / 255.0

In [111]:
flat_X_train_edstd = flatten_dataset(X_train_edstd)

In [112]:
tsne = TSNE(n_components=2)
embed_X_train_edstd = tsne.fit_transform(flat_X_train_edstd)

In [113]:
red_x = [i[0] for i in embed_X_train_edstd]
red_y = [i[1] for i in embed_X_train_edstd]


In [128]:
lab = [i for i in labels_train_ed[0]]

In [118]:
np.array(lab).shape

(1, 2095)

In [115]:
y_train_edstd = y_train_ed / 255.0
flat_y_train_edstd = flatten_dataset(y_train_edstd)


In [116]:
tsne_y = TSNE(n_components=2)
embed_y_train_edstd = tsne_y.fit_transform(flat_y_train_edstd)

In [117]:
crop_red_x = [i[0] for i in embed_y_train_edstd]
crop_red_y = [i[1] for i in embed_y_train_edstd]

In [129]:
trace = go.Scatter(
    x = red_x,
    y = red_y,
    mode = 'markers',
    marker = dict(size=2),
    text = lab
)

data = [trace]

py.iplot(data, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunil.mall/0 or inside your plot.ly account where it is named 'basic-scatter'


In [130]:
trace_crop = go.Scatter(
    x = crop_red_x,
    y = crop_red_y,
    mode = 'markers',
    marker = dict(size=2),
    text = lab
)

data_crop = [trace_crop]

py.iplot(data_crop, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunil.mall/0 or inside your plot.ly account where it is named 'basic-scatter'


In [144]:
def seg_classes(x,y,lab):
    #returns a dict with [[x_coords],[y_coords],[labels]]
    
    x_0 = []
    x_1 = []
    x_2 = []
    x_3 = []
    x_4 = []
    x_5 = []
    x_6 = []
    y_0 = []
    y_1 = []
    y_2 = []
    y_3 = []
    y_4 = []
    y_5 = []
    y_6 = []
    lab_0 = []
    lab_1 = []
    lab_2 = []
    lab_3 = []
    lab_4 = []
    lab_5 = []
    lab_6 = []
    
    for i in range(len(lab)):
        
        if lab[i] == 0:
            x_0.append(x[i])
            y_0.append(y[i])
            lab_0.append(lab[i])
        elif lab[i] == 1:
            x_1.append(x[i])
            y_1.append(y[i])
            lab_1.append(lab[i])
        elif lab[i] == 2:
            x_2.append(x[i])
            y_2.append(y[i])
            lab_2.append(lab[i])
        elif lab[i] == 3:
            x_3.append(x[i])
            y_3.append(y[i])
            lab_3.append(lab[i])
        elif lab[i] == 4:
            x_4.append(x[i])
            y_4.append(y[i])
            lab_4.append(lab[i])
        else:
            x_5.append(x[i])
            y_5.append(y[i])
            lab_5.append(lab[i])
    
    class_0 = [x_0,y_0,lab_0]
    class_1 = [x_1,y_1,lab_1]
    class_2 = [x_2,y_2,lab_2]
    class_3 = [x_3,y_3,lab_3]
    class_4 = [x_4,y_4,lab_4]
    class_5 = [x_5,y_5,lab_5]
    
    classes = {}
    
    classes[0] = class_0
    classes[1] = class_1
    classes[2] = class_2
    classes[3] = class_3
    classes[4] = class_4
    classes[5] = class_5
    
    return classes
        

In [145]:
class_dict_f = seg_classes(red_x,red_y,lab)
class_dict_c = seg_classes(crop_red_x,crop_red_y,lab)

In [155]:
def create_trace(my_dict,key, trace_name):
    
    trace = go.Scatter(
        x = my_dict[key][0],
        y = my_dict[key][1],
        mode = 'markers',
        marker = dict(size=4),
        text = my_dict[key][2],
        name = trace_name
    )
    
    return trace

def create_traces(gen_dict):
    names = {}
    names[0] = "ALB"
    names[1] = "BET"
    names[2] = "DOL"
    names[3] = "LAG"
    names[4] = "SHARK"
    names[5] = "YFT"
    
    traces = []
    for i in gen_dict:
        traces.append(create_trace(gen_dict,i, names[i]))
        
    return traces


In [156]:
traces_f = create_traces(class_dict_f)
traces_c = create_traces(class_dict_c)

data_f = [i for i in traces_f]
data_c = [i for i in traces_c]



In [157]:
py.iplot(data_f, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunil.mall/0 or inside your plot.ly account where it is named 'basic-scatter'


In [158]:
py.iplot(data_c, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunil.mall/0 or inside your plot.ly account where it is named 'basic-scatter'
